In [13]:
# increase alpha depend on differential value

import torch
import numpy as np
import output
import importlib
importlib.reload(output)

# mの正負に応じてイジングモデルのエネルギーを計算
# Not devided by N-1
def energy(m, h, J):
    m = 2 * (m > 0.0) - 1
    return -torch.sum(h * m) - (torch.sum(J * torch.outer(m, m)) - torch.sum(torch.diagonal(J * torch.outer(m, m))))

# エネルギー計算関数 (ベクトル化)
# Not devided by N-1
def f(m, h, J, tau, alpha):
    AS = -(1 - alpha) * torch.sum(torch.log(1 - m**2))
    interaction = (torch.sum(J * torch.outer(m, m)) - torch.sum(torch.diagonal(J * torch.outer(m, m)))) 
    ising = -alpha * (torch.sum(h * m) + interaction)
    return AS + ising

# 勾配計算関数 (ベクトル化)
# Not devided by N-1
def update(m, h, J, tau, alpha):
    interaction_grad = -alpha * (torch.sum(J * m, dim=1) - torch.diagonal(J) * m) 
    dif = interaction_grad + (1 - alpha) * m / (1 - m**2) - alpha * h
    return dif

# 勾配降下法 (改良版)
def gradient_descent(m, h, J, tau, alpha, alpha_inc, max_iter, lr, tol):
    best_energy = 0
    alpha_cnt = 0
    for iteration in range(max_iter):
        grad = update(m, h, J, tau, alpha)
        m = m - lr * grad  # 勾配降下ステップ
        if best_energy > energy(m, h, J):
            best_energy = energy(m, h, J)

        # 停滞のチェック（勾配が小さくなる場合）
        if torch.max(abs(grad)) < tol:
            alpha = min(alpha + alpha_inc, 0.9999)  # alpha を増加
            alpha_cnt += 1

        ## alpha が 0.99 に達した場合、終了
        #if alpha >= 0.999:
        #    break
    return energy(m, h, J), best_energy, alpha_cnt

# シード値の設定
def initialize_random_parameters(n, seed):
    torch.manual_seed(seed)
    J = torch.normal(mean=0.0, std=1.0, size=(n, n))
    return J

# 初期条件
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
seed = 42
n = 100
h = 0.0001
alpha_inc = 0.01
tau = torch.tensor(1000, device=device)
max_iter = 1000000
lr = 0.01
tol = 1e-5
final_list = []
best_list = []
alpha_cnt_list = []

for i in range(20):
    m = torch.zeros(n, device=device)
    J = initialize_random_parameters(n, seed).to(device)
    alpha = torch.tensor(0.0, device=device)

    # 勾配降下法を実行
    final_energy, best_energy, alpha_cnt = gradient_descent(m, h, J, tau, alpha, alpha_inc, max_iter, lr, tol)

    # リストに追加
    final_list.append(final_energy.cpu().item())
    best_list.append(best_energy.cpu().item())
    alpha_cnt_list.append(alpha_cnt)

    # modify seed value
    seed += 1

matrix = [[f, b, a] for f, b, a in zip(final_list, best_list, alpha_cnt_list)]
matrix.insert(0, ['final', 'best', 'alpha'])
result = output.matrix_to_csv(f'alpha{alpha_inc}_lr{lr}_tol{tol}', matrix)

# 最終結果を出力
print("Final mean energy:", np.mean(final_list))
print("Best mean energy", np.mean(best_list))


Final mean energy: -632.1508865356445
Best mean energy -772.1880859375


In [18]:
a = torch.tensor([1, -2])
print((torch.min(abs(a)).item()))

1


In [5]:
# increase alpha depend on the schedule

import torch
import numpy as np

# mの正負に応じてイジングモデルのエネルギーを計算
# Not devided by N-1
def energy(m, h, J):
    m = 2 * (m > 0.0) - 1
    return -torch.sum(h * m) - (torch.sum(J * torch.outer(m, m)) - torch.sum(torch.diagonal(J * torch.outer(m, m))))

# エネルギー計算関数 (ベクトル化)
# Not devided by N-1
def f(m, h, J, tau, alpha):
    AS = -(1 - alpha) * torch.sum(torch.log(1 - m**2))
    interaction = (torch.sum(J * torch.outer(m, m)) - torch.sum(torch.diagonal(J * torch.outer(m, m))))
    ising = -alpha * (torch.sum(h * m) + interaction)
    return AS + ising

# 勾配計算関数 (ベクトル化)
# Not devided by N-1
def update(m, h, J, tau, alpha):
    interaction_grad = -alpha * (torch.sum(J * m, dim=1) - torch.diagonal(J) * m)
    dif = interaction_grad + (1 - alpha) * m / (1 - m**2) - alpha * h
    return dif

# 勾配降下法 (改良版)
def gradient_descent(m, h, J, tau, alpha, max_iter, tol=1e-6):
    learning_rate = 0.01
    for iteration in range(max_iter):
        grad = update(m, h, J, tau, alpha)
        m = m - learning_rate * grad  # 勾配降下ステップ

        alpha = min(alpha + (1 / max_iter), 0.999999)

    print(alpha)
    return energy(m, h, J)

# シード値の設定
def initialize_random_parameters(n, seed):
    torch.manual_seed(seed)
    J = torch.normal(mean=0.0, std=1.0, size=(n, n))
    return J

# 初期条件
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
seed = 42
n = 100
h = 0.0001
tau = torch.tensor(1000, device=device)
max_iter = 1000000
final_list = []

for i in range(1):
    m = torch.zeros(n, device=device)
    J = initialize_random_parameters(n, seed).to(device)
    alpha = torch.tensor(0.0, device=device)

    # 勾配降下法を実行
    final_energy = gradient_descent(m, h, J, tau, alpha, max_iter).cpu().item()

    # リストに追加
    final_list.append(final_energy)

    # modify seed value
    seed += 1

# 最終結果を出力
print("Final mean energy:", np.mean(final_list))


0.999999
Final mean energy: -65.22675323486328
